# Extract features

We are interested in the following features:

- H1: region-averaged time-series
- H2: region-level connectivities (from H1, optional: triu-k1)
- H3: network-averaged time-series (from H1)
- H4: network connectivity (from H3, optional: triu-k1)
- H5: networks connectivity (from H2, optional: triu-k0)

> Note that we are not going to take the upper triangular part of the connectivity matrix and full matrices are used instead.

In [1]:
%reload_ext autoreload
%autoreload 2

from typing_extensions import deprecated

from src.acnets.pipeline import Parcellation, ConnectivityAggregator, \
                                ConnectivityExtractor, TimeseriesAggregator

In [2]:
@deprecated('Use full connectivity matrix instead')
def flatten(ds, mask_k=0):

    ds = ds.copy()
    import numpy as np
    node_type = ds['connectivity'].dims[-1][:-4]
    c = ds['connectivity'].values
    c[..., *np.tril_indices(c.shape[1], k=mask_k)] = np.nan
    ds['connectivity'].values = c
    c = ds['connectivity'].to_dataframe().dropna().reset_index().rename(columns={
        'connectivity': 'partial correlation'})
    c['connectivity'] = c[f'{node_type}_src'] + ' - ' + c[f'{node_type}_dst']
    c = c.pivot(index='subject', columns='connectivity', values='partial correlation')
    return c

In [3]:
atlases = [
    'dosenbach2010',
    'dosenbach2007',
    'difumo_64_2mm',
    'difumo_128_2mm',
    'difumo_256_2mm',
    'difumo_512_2mm',
    'difumo_1024_2mm',
    'gordon2014_2mm',
    'cort-maxprob-thr25-2mm',
    'seitzman2018',
    'friedman2020',
]

kinds = [
    'correlation',
    'partial correlation',
    'tangent',
    'covariance',
    'precisions'
]

h1_time_regions = Parcellation(atlas_name='dosenbach2010', verbose=1).fit_transform(X=None)
h2_conn_regions = ConnectivityExtractor(kind='partial correlation').fit_transform(h1_time_regions)
h3_time_networks = TimeseriesAggregator(strategy='network').fit_transform(h1_time_regions)
h4_conn_networks = ConnectivityExtractor(kind='partial correlation').fit_transform(h3_time_networks)
h5_conn_networks = ConnectivityAggregator(strategy='network').fit_transform(h2_conn_regions)

# TODO WIP: reshape as tidy dataframes

# h1
# h1 = h1_time_regions['timeseries'].to_dataframe().reset_index().pivot(index=['subject', 'region'], columns='timepoint')
# h1.columns = h1.columns.droplevel(0)
h1 = h1_time_regions['timeseries'].values

# h2
# h2 = flatten(h2_conn_regions, mask_k=0)
h2 = h2_conn_regions['connectivity'].values

# h3
# h3 = h3_time_networks['timeseries'].to_dataframe().reset_index().pivot(index=['subject', 'network'], columns='timepoint')
# h3.columns = h3.columns.droplevel(0)
h3 = h3_time_networks['timeseries'].values

# h4
# h4 = flatten(h4_conn_networks, mask_k=0)
h4 = h4_conn_networks['connectivity'].values

# h5
# h5 = flatten(h5_conn_networks, mask_k=-1)
h5 = h5_conn_networks['connectivity'].values
